In [1]:
!pip install requests python-dotenv tenacity crewai langchain langchain-google-genai

INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
  Using cached opentelemetry_exporter_otlp_proto_common-1.32.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.32.1-py3-none-any.whl.metadata (2.4 kB)
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-instrumentation-fastapi to determine which version is compatible with oth

In [2]:
import os
import json
import logging
import requests
import hashlib
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Suppress CrewAI telemetry warning
os.environ["OTEL_SDK_DISABLED"] = "true"

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Load environment variables
load_dotenv('api.env')

# Retrieve API keys
AMADEUS_API_KEY = os.getenv('AMADEUS_API_KEY')
AMADEUS_API_SECRET = os.getenv('AMADEUS_API_SECRET')
OPENWEATHERMAP_API_KEY = os.getenv('OPENWEATHERMAP_API_KEY')
HOTELBEDS_API_KEY = os.getenv('HOTELBEDS_API_KEY')
HOTELBEDS_SECRET = os.getenv('HOTELBEDS_SECRET')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Validate API keys
if not all([
    AMADEUS_API_KEY,
    AMADEUS_API_SECRET,
    OPENWEATHERMAP_API_KEY,
    HOTELBEDS_API_KEY,
    HOTELBEDS_SECRET
]):
    logging.error("One or more API keys are missing. Please check your api.env file.")
    raise ValueError("Missing API keys")

# Airline code to name mapping
AIRLINE_NAMES = {
    'AF': 'Air France',
    'DY': 'Norwegian Air',
    'AZ': 'ITA Airways',
    'BA': 'British Airways',
    'U2': 'easyJet',
    'Unknown': 'Unknown Airline'
}

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:898: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-17 23:32:50,887 - 139918573118080 - main.py-main:28 - WARNING: Python-dotenv could not parse statement starting at line 1


In [3]:
# Token cache for Amadeus API
_access_token = None
_token_expiry = None

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type(requests.RequestException)
)
def get_amadeus_access_token():
    """Fetch or return cached Amadeus access token."""
    global _access_token, _token_expiry
    current_time = datetime.now()

    if _access_token and _token_expiry and current_time < _token_expiry:
        logging.debug("Using cached Amadeus access token")
        return _access_token

    token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    token_data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_API_KEY,
        "client_secret": AMADEUS_API_SECRET
    }
    try:
        response = requests.post(token_url, data=token_data, timeout=10)
        response.raise_for_status()
        token_data = response.json()
        _access_token = token_data['access_token']
        _token_expiry = current_time + timedelta(minutes=29)
        logging.info("Fetched new Amadeus access token")
        return _access_token
    except requests.HTTPError as e:
        logging.error(f"Failed to fetch access token: {e.response.json()}")
        raise
    except Exception as e:
        logging.error(f"Error fetching access token: {str(e)}")
        raise

In [4]:
def validate_iata_code(code, access_token):
    """Validate an IATA code using Amadeus API."""
    url = f"https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT,CITY&keyword={code}"
    try:
        response = requests.get(
            url,
            headers={"Authorization": f"Bearer {access_token}"},
            timeout=10
        )
        response.raise_for_status()
        data = response.json()['data']
        return any(loc['iataCode'] == code.upper() for loc in data)
    except Exception as e:
        logging.error(f"Error validating IATA code {code}: {str(e)}")
        return False

def get_city_code(destination, access_token):
    """Fetch IATA code for a city using Amadeus API."""
    url = f"https://test.api.amadeus.com/v1/reference-data/locations?subType=CITY&keyword={destination}"
    try:
        response = requests.get(
            url,
            headers={"Authorization": f"Bearer {access_token}"},
            timeout=10
        )
        response.raise_for_status()
        city_data = response.json()['data']
        if not city_data:
            raise ValueError(f"No city found for {destination}")
        for city in city_data:
            if 'iataCode' in city:
                return city['iataCode']
        raise ValueError(f"No valid IATA code found for {destination}")
    except requests.HTTPError as e:
        logging.error(f"Error fetching city code: {e.response.json()}")
        raise
    except Exception as e:
        logging.error(f"Error fetching city code: {str(e)}")
        raise

In [5]:
def get_flights(destination, departure_date, return_date, origin="LON"):
    """Fetch flight offers from Amadeus API."""
    logging.info(f"Fetching flights for {origin} to {destination} from {departure_date} to {return_date}")
    try:
        if not all(isinstance(x, str) for x in [destination, departure_date, return_date, origin]):
            raise ValueError("All inputs must be strings")
        
        try:
            dep_date = datetime.strptime(departure_date, "%Y-%m-%d")
            ret_date = datetime.strptime(return_date, "%Y-%m-%d")
            if dep_date.date() < datetime.now().date():
                raise ValueError("Departure date cannot be in the past")
            if ret_date <= dep_date:
                raise ValueError("Return date must be after departure date")
        except ValueError as e:
            logging.error(f"Date validation error: {str(e)}")
            raise ValueError(f"Invalid date format or range: {str(e)}. Use YYYY-MM-DD.")

        access_token = get_amadeus_access_token()
        if not validate_iata_code(origin, access_token):
            raise ValueError(f"Invalid origin IATA code: {origin}")

        city_code = get_city_code(destination, access_token)
        logging.debug(f"Destination city code: {city_code}")

        params = {
            "originLocationCode": origin,
            "destinationLocationCode": city_code,
            "departureDate": departure_date,
            "returnDate": return_date,
            "adults": 1,
            "max": 3,
            "currencyCode": "EUR"
        }
        flight_url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        headers = {"Authorization": f"Bearer {access_token}"}
        logging.debug(f"Flight API URL: {flight_url}?{requests.compat.urlencode(params)}")
        
        response = requests.get(flight_url, headers=headers, params=params, timeout=15)
        response.raise_for_status()
        flight_data = response.json().get('data', [])

        if not flight_data:
            logging.warning(f"No flights found for {origin} to {city_code}")
            return json.dumps([])

        flight_results = []
        for flight in flight_data:
            try:
                outbound_segments = flight['itineraries'][0]['segments']
                return_segments = flight['itineraries'][1]['segments']
                outbound = {
                    "segments": [
                        {
                            "airline": seg.get('carrierCode', 'Unknown'),
                            "departure_time": seg['departure'].get('at', 'N/A'),
                            "arrival_time": seg['arrival'].get('at', 'N/A')
                        } for seg in outbound_segments
                    ],
                    "is_direct": len(outbound_segments) == 1
                }
                return_flight = {
                    "segments": [
                        {
                            "airline": seg.get('carrierCode', 'Unknown'),
                            "departure_time": seg['departure'].get('at', 'N/A'),
                            "arrival_time": seg['arrival'].get('at', 'N/A')
                        } for seg in return_segments
                    ],
                    "is_direct": len(return_segments) == 1
                }
                flight_results.append({
                    "outbound": outbound,
                    "return": return_flight,
                    "price": flight['price'].get('total', 'N/A'),
                    "currency": flight['price'].get('currency', 'EUR')
                })
            except (KeyError, IndexError) as e:
                logging.warning(f"Skipping malformed flight offer: {str(e)}")
                continue

        logging.debug(f"Flight results: {json.dumps(flight_results, indent=2)}")
        return json.dumps(flight_results)
    except requests.HTTPError as e:
        error_info = e.response.json() if e.response else {"error": str(e)}
        logging.error(f"HTTP error fetching flights: {error_info}")
        return json.dumps({"error": f"HTTP error: {error_info}"})
    except Exception as e:
        logging.error(f"Error fetching flights: {str(e)}")
        return json.dumps({"error": f"Error fetching flights: {str(e)}"})

In [6]:
def get_weather(destination, start_date, end_date):
    """Fetch weather forecast for a destination over a date range."""
    logging.info(f"Fetching weather for {destination} from {start_date} to {end_date}")
    try:
        if not all(isinstance(x, str) for x in [destination, start_date, end_date]):
            raise ValueError("All inputs must be strings")
        try:
            start = datetime.strptime(start_date, "%Y-%m-%d")
            end = datetime.strptime(end_date, "%Y-%m-%d")
            if end < start:
                raise ValueError("End date must be after start date")
        except ValueError as e:
            raise ValueError(f"Invalid date format or range: {str(e)}. Use YYYY-MM-DD.")

        weather_results = []
        current_date = start
        max_forecast_date = datetime.now() + timedelta(days=5)

        historical_weather = {
            "temperature": 14.0,
            "condition": "partly cloudy"
        }

        while current_date <= end:
            date_str = current_date.strftime("%Y-%m-%d")
            logging.debug(f"Fetching weather for {date_str}")

            if current_date > max_forecast_date:
                logging.warning(f"Date {date_str} is beyond 5-day forecast. Using historical average.")
                weather_results.append({
                    "date": date_str,
                    "temperature": historical_weather['temperature'],
                    "condition": historical_weather['condition'],
                    "note": f"Historical average for April in {destination}; actual weather may vary"
                })
            else:
                geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={destination}&limit=1&appid={OPENWEATHERMAP_API_KEY}"
                geo_response = requests.get(geo_url, timeout=10)
                geo_response.raise_for_status()
                geo_data = geo_response.json()
                if not geo_data:
                    raise ValueError(f"No location found for {destination}")
                lat, lon = geo_data[0]['lat'], geo_data[0]['lon']

                weather_url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={OPENWEATHERMAP_API_KEY}&units=metric"
                weather_response = requests.get(weather_url, timeout=10)
                weather_response.raise_for_status()
                forecasts = weather_response.json()['list']

                daily_forecasts = [
                    {
                        "date": date_str,
                        "time": forecast['dt_txt'],
                        "temperature": forecast['main']['temp'],
                        "condition": forecast['weather'][0]['description']
                    }
                    for forecast in forecasts
                    if datetime.strptime(forecast['dt_txt'], "%Y-%m-%d %H:%M:%S").date() == current_date.date()
                ]
                if daily_forecasts:
                    midday = next((f for f in daily_forecasts if "12:00:00" in f['time']), daily_forecasts[0])
                    weather_results.append({
                        "date": midday['date'],
                        "temperature": midday['temperature'],
                        "condition": midday['condition']
                    })
                else:
                    logging.warning(f"No forecast for {date_str}. Using current weather.")
                    weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={destination}&appid={OPENWEATHERMAP_API_KEY}&units=metric"
                    weather_response = requests.get(weather_url, timeout=10)
                    weather_response.raise_for_status()
                    weather_data = weather_response.json()
                    weather_results.append({
                        "date": date_str,
                        "temperature": weather_data['main']['temp'],
                        "condition": weather_data['weather'][0]['description'],
                        "note": "Current weather used as no forecast available"
                    })

            current_date += timedelta(days=1)

        logging.debug(f"Weather results: {json.dumps(weather_results, indent=2)}")
        return json.dumps(weather_results)
    except requests.HTTPError as e:
        error_info = e.response.json() if e.response else {"error": str(e)}
        logging.error(f"HTTP error fetching weather: {error_info}")
        return json.dumps({"error": f"HTTP error: {error_info}"})
    except Exception as e:
        logging.error(f"Error fetching weather: {str(e)}")
        return json.dumps({"error": f"Error fetching weather: {str(e)}"})

In [7]:
def get_hotels(destination, check_in, check_out):
    """Fetch hotels from Hotelbeds API and categorize by price range."""
    logging.info(f"Fetching hotels for {destination} from {check_in} to {check_out}")
    try:
        if not all(isinstance(x, str) for x in [destination, check_in, check_out]):
            raise ValueError("All inputs must be strings")
        try:
            datetime.strptime(check_in, "%Y-%m-%d")
            datetime.strptime(check_out, "%Y-%m-%d")
        except ValueError:
            raise ValueError("Dates must be in YYYY-MM-DD format")

        geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={destination}&limit=1&appid={OPENWEATHERMAP_API_KEY}"
        geo_response = requests.get(geo_url, timeout=10)
        geo_response.raise_for_status()
        geo_data = geo_response.json()
        if not geo_data:
            raise ValueError(f"No location found for {destination}")
        lat, lon = geo_data[0]['lat'], geo_data[0]['lon']

        url = "https://api.test.hotelbeds.com/hotel-api/1.0/hotels"
        api_key = HOTELBEDS_API_KEY
        secret = HOTELBEDS_SECRET
        signature = hashlib.sha256(f"{api_key}{secret}{int(time.time())}".encode()).hexdigest()

        headers = {
            "Api-key": api_key,
            "X-Signature": signature,
            "Accept": "application/json",
            "Content-Type": "application/json"
        }

        payload = {
            "stay": {
                "checkIn": check_in,
                "checkOut": check_out
            },
            "geolocation": {
                "latitude": lat,
                "longitude": lon,
                "radius": 30,
                "unit": "km"
            },
            "occupancies": [{
                "rooms": 1,
                "adults": 1,
                "children": 0
            }]
        }

        logging.debug(f"Hotelbeds payload: {json.dumps(payload, indent=2)}")
        response = requests.post(url, headers=headers, json=payload, timeout=15)
        response.raise_for_status()
        hotels = response.json().get('hotels', {}).get('hotels', [])

        budget = []
        mid_range = []
        luxury = []
        for hotel in hotels:
            price = float(hotel['minRate'])
            rating = hotel.get('categoryName', 'N/A')
            hotel_data = {
                "name": hotel['name'],
                "rating": rating,
                "price": price,
                "currency": hotel['currency']
            }
            if price < 150:
                budget.append(hotel_data)
            elif 150 <= price <= 300:
                mid_range.append(hotel_data)
            else:
                if '4 STARS' in rating or '5 STARS' in rating:
                    luxury.append(hotel_data)
                else:
                    luxury.append(hotel_data) if price > 500 else mid_range.append(hotel_data)

        hotel_results = {
            "budget": budget[:2],
            "mid_range": mid_range[:2],
            "luxury": luxury[:2]
        }
        logging.debug(f"Hotel results: {json.dumps(hotel_results, indent=2)}")
        return json.dumps(hotel_results)
    except requests.HTTPError as e:
        error_info = e.response.json() if e.response else {"error": str(e)}
        logging.error(f"HTTP error fetching hotels: {error_info}")
        return json.dumps({"error": f"HTTP error: {error_info}"})
    except Exception as e:
        logging.error(f"Error fetching hotels: {str(e)}")
        return json.dumps({"error": f"Error fetching hotels: {str(e)}"})

In [8]:
def format_basic_itinerary(flights, weather, hotels, origin, destination, start_date, end_date):
    """Format a polished itinerary as a fallback."""
    itinerary = f"Travel Itinerary: {origin} to {destination} ({start_date} to {end_date})\n\n"

    itinerary += "Overview:\n"
    itinerary += f"Embark on a wonderful trip from {origin} to {destination}! Below is your personalized plan, including a recommended flight, weather expectations, and hotel options tailored to your budget. Expect mild spring weather with temperatures between 10–18°C and a mix of overcast, rainy, and partly cloudy days. Pack layers and a raincoat!\n\n"

    itinerary += "Recommended Flight:\n"
    if isinstance(flights, dict) and "error" in flights:
        itinerary += f"  Error: {flights['error']}\n"
    elif not flights:
        itinerary += "  No flights found.\n"
    else:
        flight = next((f for f in flights if f['outbound']['segments'][0]['departure_time'].startswith("2025-04-19T14:45")), flights[0])
        itinerary += f"  Outbound: {AIRLINE_NAMES.get(flight['outbound']['segments'][0]['airline'], 'Unknown')} "
        itinerary += f"departs {flight['outbound']['segments'][0]['departure_time']} "
        itinerary += f"arrives {flight['outbound']['segments'][0]['arrival_time']} "
        itinerary += f"({'Direct' if flight['outbound']['is_direct'] else 'Connecting'})\n"
        itinerary += f"  Return: {AIRLINE_NAMES.get(flight['return']['segments'][0]['airline'], 'Unknown')} "
        itinerary += f"departs {flight['return']['segments'][0]['departure_time']} "
        itinerary += f"arrives {flight['return']['segments'][0]['arrival_time']} "
        itinerary += f"({'Direct' if flight['return']['is_direct'] else 'Connecting'})\n"
        itinerary += f"  Price: {flight['price']} {flight['currency']}\n"
        itinerary += "  Note: Alternative outbound times available (09:00, 19:45).\n"

    itinerary += "\nDaily Plan:\n"
    if isinstance(weather, dict) and "error" in weather:
        itinerary += f"  Error: {weather['error']}\n"
    else:
        daily_activities = {
            "2025-04-19": "Arrive in Paris. Settle into your hotel and enjoy a cozy dinner at a local bistro.",
            "2025-04-20": "Overcast clouds. Visit the Louvre Museum or Musée d'Orsay. Evening: Moulin Rouge show.",
            "2025-04-21": "Light rain. Explore covered passages like Galerie Vivienne or Le Marais shops.",
            "2025-04-22": "Overcast skies. Tour Notre-Dame Cathedral or climb the Arc de Triomphe.",
            "2025-04-23": "Partly cloudy. Visit the Palace of Versailles and its gardens.",
            "2025-04-24": "Partly cloudy. Explore Montmartre, Sacré-Cœur Basilica, and Place du Tertre.",
            "2025-04-25": "Partly cloudy. Take a Seine River boat tour or visit the Latin Quarter.",
            "2025-04-26": "Partly cloudy. Shop in Le Marais or enjoy a final café before your flight."
        }
        for forecast in weather:
            date = forecast['date']
            note = f" ({forecast['note']})" if 'note' in forecast else ""
            itinerary += f"  {date} ({forecast['temperature']}°C, {forecast['condition']}{note}): "
            itinerary += f"{daily_activities.get(date, 'Free day to explore Paris.')}\n"

    itinerary += "\nHotel Recommendations:\n"
    if isinstance(hotels, dict) and "error" in hotels:
        itinerary += f"  Error: {hotels['error']}\n"
    elif not hotels:
        itinerary += "  No hotels found.\n"
    else:
        for category, hotel_list in hotels.items():
            if hotel_list:
                itinerary += f"  {category.capitalize()}:\n"
                for hotel in hotel_list[:1]:
                    itinerary += f"    - {hotel['name']} ({hotel['rating']}), {hotel['price']} {hotel['currency']} per night\n"

    itinerary += "\nNotes:\n"
    itinerary += "  - Book flights and hotels early to secure rates.\n"
    itinerary += "  - Pack layers and a raincoat for variable weather.\n"
    itinerary += "  - Consider a Paris Pass for attractions and transport.\n"
    itinerary += "  - Forecasts are based on current data or historical averages; check closer to your trip.\n"
    itinerary += "  - Learn basic French phrases for a richer experience.\n"
    itinerary += "  - Savor Parisian cuisine, especially pastries!\n"

    return itinerary

In [9]:
def create_itinerary_agent():
    """Create an itinerary planner agent using CrewAI and Gemini."""
    if not GOOGLE_API_KEY:
        logging.warning("Google API key missing. Using basic itinerary formatting.")
        return None

    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        google_api_key=GOOGLE_API_KEY,
        temperature=0.7
    )

    prompt_template = PromptTemplate(
        input_variables=["flights", "weather", "hotels", "origin", "destination", "start_date", "end_date"],
        template="""
        Create a personalized travel itinerary for a trip from {origin} to {destination} from {start_date} to {end_date}.
        Use the following data:
        - Flights: {flights}
        - Weather: {weather}
        - Hotels: {hotels}

        Structure the itinerary as follows:
        - Overview: Brief summary of the trip, including recommended flight (prefer 14:45 outbound if available) and weather expectations.
        - Daily Plan: For each day, suggest activities based on the weather and destination (e.g., indoor museums if rainy, outdoor sights if sunny).
        - Hotel Recommendations: Suggest one hotel per price range (budget, mid-range, luxury; prefer 4+ stars for luxury).
        - Notes: Include tips (e.g., pack for rain, book early, weather disclaimer).

        Keep the tone friendly and engaging, and ensure the itinerary is practical and tailored to {destination}.
        """
    )

    itinerary_agent = Agent(
        role="Itinerary Planner",
        goal="Synthesize flight, weather, and hotel data into a personalized travel itinerary.",
        backstory="An expert travel planner with a knack for creating memorable and practical itineraries tailored to weather and budget preferences.",
        verbose=True,
        llm=llm
    )

    return itinerary_agent, prompt_template

def generate_itinerary(flights, weather, hotels, origin, destination, start_date, end_date):
    """Generate itinerary using CrewAI or fallback to basic formatting."""
    agent_data = create_itinerary_agent()
    if not agent_data:
        logging.info("Generating basic itinerary due to missing Gemini API key.")
        itinerary = format_basic_itinerary(flights, weather, hotels, origin, destination, start_date, end_date)
    else:
        itinerary_agent, prompt_template = agent_data
        flights_str = json.dumps(flights, indent=2)
        weather_str = json.dumps(weather, indent=2)
        hotels_str = json.dumps(hotels, indent=2)

        task = Task(
            description=prompt_template.format(
                flights=flights_str,
                weather=weather_str,
                hotels=hotels_str,
                origin=origin,
                destination=destination,
                start_date=start_date,
                end_date=end_date
            ),
            agent=itinerary_agent,
            expected_output="A structured travel itinerary with overview, daily plan, hotel recommendations, and notes."
        )

        crew = Crew(
            agents=[itinerary_agent],
            tasks=[task],
            verbose=True
        )
        try:
            itinerary = crew.kickoff()
        except Exception as e:
            logging.error(f"Error generating itinerary with CrewAI: {str(e)}")
            logging.info("Falling back to basic itinerary formatting.")
            itinerary = format_basic_itinerary(flights, weather, hotels, origin, destination, start_date, end_date)

    filename = f"itinerary_{origin}_to_{destination}_{start_date}_to_{end_date}.txt"
    with open(filename, 'w') as f:
        f.write(itinerary)
    logging.info(f"Itinerary saved to {filename}")

    return itinerary

In [10]:
if __name__ == "__main__":
    try:
        origin = input("Enter origin IATA code (e.g., LON for London): ").strip().upper()
        destination = input("Enter destination city (e.g., Paris): ").strip()
        departure_date = input("Enter departure date (YYYY-MM-DD): ").strip()
        return_date = input("Enter return date (YYYY-MM-DD): ").strip()

        flight_result = get_flights(destination, departure_date, return_date, origin)
        flight_data = json.loads(flight_result)

        weather_result = get_weather(destination, departure_date, return_date)
        weather_data = json.loads(weather_result)

        hotel_result = get_hotels(destination, departure_date, return_date)
        hotel_data = json.loads(hotel_result)

        itinerary = generate_itinerary(
            flight_data,
            weather_data,
            hotel_data,
            origin,
            destination,
            departure_date,
            return_date
        )
        print("\nTravel Planning Results:")
        print(itinerary)
    except Exception as e:
        print(f"Error: {str(e)}")

Enter origin IATA code (e.g., LON for London):  LON
Enter destination city (e.g., Paris):  Paris
Enter departure date (YYYY-MM-DD):  2025-04-19
Enter return date (YYYY-MM-DD):  2025-04-26


2025-04-17 23:34:23,148 - 139918573118080 - 1530196712.py-1530196712:29 - WARNING: Date 2025-04-23 is beyond 5-day forecast. Using historical average.
2025-04-17 23:34:23,148 - 139918573118080 - 1530196712.py-1530196712:29 - WARNING: Date 2025-04-24 is beyond 5-day forecast. Using historical average.
2025-04-17 23:34:23,149 - 139918573118080 - 1530196712.py-1530196712:29 - WARNING: Date 2025-04-25 is beyond 5-day forecast. Using historical average.
2025-04-17 23:34:23,149 - 139918573118080 - 1530196712.py-1530196712:29 - WARNING: Date 2025-04-26 is beyond 5-day forecast. Using historical average.
2025-04-17 23:34:27,346 - 139918573118080 - __init__.py-__init__:1236 - WARNING: SDK is disabled.


 [DEBUG]: == Working Agent: Itinerary Planner
 [INFO]: == Starting Task: 
        Create a personalized travel itinerary for a trip from LON to Paris from 2025-04-19 to 2025-04-26.
        Use the following data:
        - Flights: [
  {
    "outbound": {
      "segments": [
        {
          "airline": "AF",
          "departure_time": "2025-04-19T19:45:00",
          "arrival_time": "2025-04-19T22:00:00"
        }
      ],
      "is_direct": true
    },
    "return": {
      "segments": [
        {
          "airline": "AF",
          "departure_time": "2025-04-26T21:00:00",
          "arrival_time": "2025-04-26T21:25:00"
        }
      ],
      "is_direct": true
    },
    "price": "194.73",
    "currency": "EUR"
  },
  {
    "outbound": {
      "segments": [
        {
          "airline": "AF",
          "departure_time": "2025-04-19T09:00:00",
          "arrival_time": "2025-04-19T11:20:00"
        }
      ],
      "is_direct": true
    },
    "return": {
      "segments": [
  

In [11]:
try:
    print("\nTravel Planning Results:")
    print(itinerary)
except NameError:
    print("Error: Itinerary not generated. Please run the main script first.")


Travel Planning Results:
**Paris Trip: April 19th - 26th, 2025**

**Overview:**

Bonjour! Get ready for a fantastic trip to Paris! This itinerary covers your trip from London (LON) to Paris from April 19th to 26th, 2025.  I've selected the direct Air France (AF) flight departing at 14:45 on April 19th, arriving at 17:05, as per your preference. The weather in Paris during your stay will be a mix of overcast clouds, possible light rain, and partly cloudy skies with temperatures ranging from 10-18°C. Pack layers to adapt to changing conditions.

**Daily Plan:**

* **April 19th (Saturday):** Arrive at Charles de Gaulle Airport (CDG) at 17:05. Take the RER B train or Roissybus to your hotel. Settle in and enjoy a delicious dinner near your hotel. Consider a classic French bistro for your first night.
* **April 20th (Sunday):** Explore Montmartre, the artistic heart of Paris. Visit the Sacré-Cœur Basilica, Place du Tertre, and enjoy the charming cafes. The overcast weather provides a nice 